In [1]:
#참고링크 https://github.com/allenai/bilm-tf
#참고링크 https://appliedmachinelearning.wordpress.com/2019/11/30/training-elmo-from-scratch-on-custom-data-set-for-generating-embeddings-tensorflow/

elem 환경에서 실행

In [1]:
#이 코드는 실행하면 cpu로 연산하기 때문에 느리므로 훈련데이터 12935번째 문서를 임베딩 할 때에만 커널을 다시 시작 후 실행
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

C:\Users\user\anaconda3\envs\elem\lib\site-packages\tensorflow\python\framework\dtypes.py:458: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\user\anaconda3\envs\elem\lib\site-packages\tensorflow\python\framework\dtypes.py:459: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\user\anaconda3\envs\elem\lib\site-packages\tensorflow\python\framework\dtypes.py:460: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\user\anaconda3\envs\elem\lib\site-packages\tensorflow\python\framework

In [2]:
import pickle

# 피클 파일에서 리스트 불러오기
with open('to_txt.pkl', 'rb') as f:
    to_txt = pickle.load(f)

In [3]:
len(to_txt) #전처리 된 reuters news 데이터 문서 리스트

9762

In [4]:
x_train=to_txt[:7828]
x_test=to_txt[7828:]

엘모벡터 생성

In [7]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
import numpy as np
import scipy.spatial.distance as ds
from bilm import Batcher, BidirectionalLanguageModel, weight_layers
 
# Location of pretrained LM.  Here we use the test fixtures.
datadir = os.path.join('C:/Users/user/Desktop/bilm-tf-master/swb', 'model(reuters256)') #사전 학습된 모델이 있는 디렉토리
vocab_file = os.path.join(datadir, 'vocab.txt')
options_file = os.path.join(datadir, 'options.json')
weight_file = os.path.join(datadir, 'swb_weights.hdf5')
 
# Create a Batcher to map text to character ids.
batcher = Batcher(vocab_file, 50)
 
# Input placeholders to the biLM.
context_character_ids = tf.placeholder('int32', shape=(None, None, 50))
 
# Build the biLM graph.
bilm = BidirectionalLanguageModel(options_file, weight_file)
 
# Get ops to compute the LM embeddings.
context_embeddings_op = bilm(context_character_ids)
 
# Get an op to compute ELMo (weighted average of the internal biLM layers)
elmo_context_input = weight_layers('input', context_embeddings_op, l2_coef=0.0)
 
# Now we can compute embeddings.
raw_context = ['Technology has advanced so much in new scientific world',
                'My child participated in fancy dress competition',
                'Fashion industry has seen tremendous growth in new designs']
 
tokenized_context = [sentence.split() for sentence in raw_context]
print(tokenized_context)

USING SKIP CONNECTIONS
[['Technology', 'has', 'advanced', 'so', 'much', 'in', 'new', 'scientific', 'world'], ['My', 'child', 'participated', 'in', 'fancy', 'dress', 'competition'], ['Fashion', 'industry', 'has', 'seen', 'tremendous', 'growth', 'in', 'new', 'designs']]


In [8]:
with tf.Session() as sess:
    # It is necessary to initialize variables once before running inference.
    sess.run(tf.global_variables_initializer())
 
    # Create batches of data.
    context_ids = batcher.batch_sentences(tokenized_context)
    print("Shape of context ids = ", context_ids.shape)
 
    # Compute ELMo representations (here for the input only, for simplicity).
    elmo_context_input_ = sess.run(
        elmo_context_input['weighted_op'],
        feed_dict={context_character_ids: context_ids}
    )
 
print("Shape of generated embeddings = ",elmo_context_input_.shape)

Shape of context ids =  (3, 11, 50)
Shape of generated embeddings =  (3, 9, 256)


In [9]:
# Computing euclidean distance between words embedding
euc_dist_bet_tech_computer = np.linalg.norm(elmo_context_input_[1,5,:]-elmo_context_input_[0,0,:])
euc_dist_bet_computer_fashion = np.linalg.norm(elmo_context_input_[1,5,:]-elmo_context_input_[2,0,:])
# Computing cosine distance between words embedding
cos_dist_bet_tech_computer = ds.cosine(elmo_context_input_[1,5,:],elmo_context_input_[0,0,:])
cos_dist_bet_computer_fashion = ds.cosine(elmo_context_input_[1,5,:],elmo_context_input_[2,0,:])
 
print("Euclidean Distance Comparison - ")
print("\nDress-Technology = ",np.round(euc_dist_bet_tech_computer,2),"\nDress-Fashion = ",
      np.round(euc_dist_bet_computer_fashion,2))
print("\n\nCosine Distance Comparison - ")
print("\nDress-Technology = ",np.round(cos_dist_bet_tech_computer,2),"\nDress-Fashion = ",
      np.round(cos_dist_bet_computer_fashion,2))

Euclidean Distance Comparison - 

Dress-Technology =  17.36 
Dress-Fashion =  16.23


Cosine Distance Comparison - 

Dress-Technology =  0.81 
Dress-Fashion =  0.76


임베딩

In [10]:
except_lst=[]

In [11]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
import numpy as np
import scipy.spatial.distance as ds
from bilm import Batcher, BidirectionalLanguageModel, weight_layers

for i in range(7199,len(x_train)):
    try:
        with tf.Session() as sess:

            sess.run(tf.global_variables_initializer())
            context_ids = batcher.batch_sentences([x_train[i].split()])
            elmo_context_input_ = sess.run(elmo_context_input['weighted_op'],feed_dict={context_character_ids: context_ids}) 
        print("Shape of generated embeddings = ",elmo_context_input_.shape)
        np.save('./elmo_embedding/train(reuters256)/doc%d'%i,elmo_context_input_[0])
    except:
        except_lst.append(i)

Shape of generated embeddings =  (1, 450, 256)
Shape of generated embeddings =  (1, 62, 256)
Shape of generated embeddings =  (1, 293, 256)
Shape of generated embeddings =  (1, 210, 256)
Shape of generated embeddings =  (1, 89, 256)
Shape of generated embeddings =  (1, 103, 256)
Shape of generated embeddings =  (1, 55, 256)
Shape of generated embeddings =  (1, 184, 256)
Shape of generated embeddings =  (1, 70, 256)
Shape of generated embeddings =  (1, 70, 256)
Shape of generated embeddings =  (1, 87, 256)
Shape of generated embeddings =  (1, 25, 256)
Shape of generated embeddings =  (1, 100, 256)
Shape of generated embeddings =  (1, 45, 256)
Shape of generated embeddings =  (1, 16, 256)
Shape of generated embeddings =  (1, 52, 256)
Shape of generated embeddings =  (1, 16, 256)
Shape of generated embeddings =  (1, 85, 256)
Shape of generated embeddings =  (1, 43, 256)
Shape of generated embeddings =  (1, 101, 256)
Shape of generated embeddings =  (1, 72, 256)
Shape of generated embeddin

Shape of generated embeddings =  (1, 38, 256)
Shape of generated embeddings =  (1, 107, 256)
Shape of generated embeddings =  (1, 159, 256)
Shape of generated embeddings =  (1, 50, 256)
Shape of generated embeddings =  (1, 15, 256)
Shape of generated embeddings =  (1, 84, 256)
Shape of generated embeddings =  (1, 67, 256)
Shape of generated embeddings =  (1, 48, 256)
Shape of generated embeddings =  (1, 48, 256)
Shape of generated embeddings =  (1, 25, 256)
Shape of generated embeddings =  (1, 99, 256)
Shape of generated embeddings =  (1, 80, 256)
Shape of generated embeddings =  (1, 84, 256)
Shape of generated embeddings =  (1, 164, 256)
Shape of generated embeddings =  (1, 27, 256)
Shape of generated embeddings =  (1, 76, 256)
Shape of generated embeddings =  (1, 15, 256)
Shape of generated embeddings =  (1, 20, 256)
Shape of generated embeddings =  (1, 71, 256)
Shape of generated embeddings =  (1, 20, 256)
Shape of generated embeddings =  (1, 50, 256)
Shape of generated embeddings =

Shape of generated embeddings =  (1, 50, 256)
Shape of generated embeddings =  (1, 167, 256)
Shape of generated embeddings =  (1, 94, 256)
Shape of generated embeddings =  (1, 57, 256)
Shape of generated embeddings =  (1, 16, 256)
Shape of generated embeddings =  (1, 74, 256)
Shape of generated embeddings =  (1, 80, 256)
Shape of generated embeddings =  (1, 125, 256)
Shape of generated embeddings =  (1, 229, 256)
Shape of generated embeddings =  (1, 115, 256)
Shape of generated embeddings =  (1, 26, 256)
Shape of generated embeddings =  (1, 16, 256)
Shape of generated embeddings =  (1, 172, 256)
Shape of generated embeddings =  (1, 94, 256)
Shape of generated embeddings =  (1, 62, 256)
Shape of generated embeddings =  (1, 46, 256)
Shape of generated embeddings =  (1, 457, 256)
Shape of generated embeddings =  (1, 45, 256)
Shape of generated embeddings =  (1, 103, 256)
Shape of generated embeddings =  (1, 67, 256)
Shape of generated embeddings =  (1, 27, 256)
Shape of generated embeddin

Shape of generated embeddings =  (1, 77, 256)
Shape of generated embeddings =  (1, 467, 256)
Shape of generated embeddings =  (1, 96, 256)
Shape of generated embeddings =  (1, 58, 256)
Shape of generated embeddings =  (1, 89, 256)
Shape of generated embeddings =  (1, 232, 256)
Shape of generated embeddings =  (1, 314, 256)
Shape of generated embeddings =  (1, 18, 256)
Shape of generated embeddings =  (1, 83, 256)
Shape of generated embeddings =  (1, 20, 256)
Shape of generated embeddings =  (1, 174, 256)
Shape of generated embeddings =  (1, 110, 256)
Shape of generated embeddings =  (1, 94, 256)
Shape of generated embeddings =  (1, 90, 256)
Shape of generated embeddings =  (1, 186, 256)
Shape of generated embeddings =  (1, 68, 256)
Shape of generated embeddings =  (1, 121, 256)
Shape of generated embeddings =  (1, 98, 256)
Shape of generated embeddings =  (1, 72, 256)
Shape of generated embeddings =  (1, 86, 256)
Shape of generated embeddings =  (1, 50, 256)
Shape of generated embeddin

In [12]:
#12935번째 문서는 길이가 너무 커서 GPU가 아닌 CPU로 실행하여 임베딩 벡터를 저장할 것, 첫번째 코드를 실행해서 CPU로 바꿔 실행 가능
except_lst

[]

In [13]:
except_test_lst=[]

In [14]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
import numpy as np
import scipy.spatial.distance as ds
from bilm import Batcher, BidirectionalLanguageModel, weight_layers

for i in range(len(x_test)):
    try:
        with tf.Session() as sess:

            sess.run(tf.global_variables_initializer())
            context_ids = batcher.batch_sentences([x_test[i].split()])
            elmo_context_input_ = sess.run(elmo_context_input['weighted_op'],feed_dict={context_character_ids: context_ids}) 
        print("Shape of generated embeddings = ",elmo_context_input_.shape)
        np.save('./elmo_embedding/test(reuters256)/test%d'%i,elmo_context_input_[0])
    except:
        except_test_lst.append(i)

Shape of generated embeddings =  (1, 144, 256)
Shape of generated embeddings =  (1, 744, 256)
Shape of generated embeddings =  (1, 227, 256)
Shape of generated embeddings =  (1, 171, 256)
Shape of generated embeddings =  (1, 186, 256)
Shape of generated embeddings =  (1, 79, 256)
Shape of generated embeddings =  (1, 248, 256)
Shape of generated embeddings =  (1, 117, 256)
Shape of generated embeddings =  (1, 122, 256)
Shape of generated embeddings =  (1, 206, 256)
Shape of generated embeddings =  (1, 59, 256)
Shape of generated embeddings =  (1, 205, 256)
Shape of generated embeddings =  (1, 26, 256)
Shape of generated embeddings =  (1, 376, 256)
Shape of generated embeddings =  (1, 702, 256)
Shape of generated embeddings =  (1, 26, 256)
Shape of generated embeddings =  (1, 78, 256)
Shape of generated embeddings =  (1, 46, 256)
Shape of generated embeddings =  (1, 102, 256)
Shape of generated embeddings =  (1, 72, 256)
Shape of generated embeddings =  (1, 21, 256)
Shape of generated em

Shape of generated embeddings =  (1, 89, 256)
Shape of generated embeddings =  (1, 51, 256)
Shape of generated embeddings =  (1, 67, 256)
Shape of generated embeddings =  (1, 80, 256)
Shape of generated embeddings =  (1, 182, 256)
Shape of generated embeddings =  (1, 16, 256)
Shape of generated embeddings =  (1, 96, 256)
Shape of generated embeddings =  (1, 16, 256)
Shape of generated embeddings =  (1, 48, 256)
Shape of generated embeddings =  (1, 81, 256)
Shape of generated embeddings =  (1, 363, 256)
Shape of generated embeddings =  (1, 77, 256)
Shape of generated embeddings =  (1, 43, 256)
Shape of generated embeddings =  (1, 150, 256)
Shape of generated embeddings =  (1, 63, 256)
Shape of generated embeddings =  (1, 154, 256)
Shape of generated embeddings =  (1, 50, 256)
Shape of generated embeddings =  (1, 147, 256)
Shape of generated embeddings =  (1, 98, 256)
Shape of generated embeddings =  (1, 30, 256)
Shape of generated embeddings =  (1, 178, 256)
Shape of generated embedding

Shape of generated embeddings =  (1, 245, 256)
Shape of generated embeddings =  (1, 116, 256)
Shape of generated embeddings =  (1, 202, 256)
Shape of generated embeddings =  (1, 109, 256)
Shape of generated embeddings =  (1, 16, 256)
Shape of generated embeddings =  (1, 37, 256)
Shape of generated embeddings =  (1, 172, 256)
Shape of generated embeddings =  (1, 156, 256)
Shape of generated embeddings =  (1, 67, 256)
Shape of generated embeddings =  (1, 78, 256)
Shape of generated embeddings =  (1, 165, 256)
Shape of generated embeddings =  (1, 196, 256)
Shape of generated embeddings =  (1, 24, 256)
Shape of generated embeddings =  (1, 107, 256)
Shape of generated embeddings =  (1, 51, 256)
Shape of generated embeddings =  (1, 93, 256)
Shape of generated embeddings =  (1, 174, 256)
Shape of generated embeddings =  (1, 16, 256)
Shape of generated embeddings =  (1, 93, 256)
Shape of generated embeddings =  (1, 314, 256)
Shape of generated embeddings =  (1, 24, 256)
Shape of generated embe

Shape of generated embeddings =  (1, 245, 256)
Shape of generated embeddings =  (1, 93, 256)
Shape of generated embeddings =  (1, 25, 256)
Shape of generated embeddings =  (1, 551, 256)
Shape of generated embeddings =  (1, 135, 256)
Shape of generated embeddings =  (1, 66, 256)
Shape of generated embeddings =  (1, 55, 256)
Shape of generated embeddings =  (1, 189, 256)
Shape of generated embeddings =  (1, 253, 256)
Shape of generated embeddings =  (1, 169, 256)
Shape of generated embeddings =  (1, 57, 256)
Shape of generated embeddings =  (1, 64, 256)
Shape of generated embeddings =  (1, 24, 256)
Shape of generated embeddings =  (1, 114, 256)
Shape of generated embeddings =  (1, 77, 256)
Shape of generated embeddings =  (1, 83, 256)
Shape of generated embeddings =  (1, 40, 256)
Shape of generated embeddings =  (1, 192, 256)
Shape of generated embeddings =  (1, 47, 256)
Shape of generated embeddings =  (1, 99, 256)
Shape of generated embeddings =  (1, 687, 256)
Shape of generated embedd

Shape of generated embeddings =  (1, 66, 256)
Shape of generated embeddings =  (1, 100, 256)
Shape of generated embeddings =  (1, 81, 256)
Shape of generated embeddings =  (1, 282, 256)
Shape of generated embeddings =  (1, 271, 256)
Shape of generated embeddings =  (1, 350, 256)
Shape of generated embeddings =  (1, 78, 256)
Shape of generated embeddings =  (1, 42, 256)
Shape of generated embeddings =  (1, 189, 256)
Shape of generated embeddings =  (1, 171, 256)
Shape of generated embeddings =  (1, 21, 256)
Shape of generated embeddings =  (1, 38, 256)
Shape of generated embeddings =  (1, 45, 256)
Shape of generated embeddings =  (1, 16, 256)
Shape of generated embeddings =  (1, 72, 256)
Shape of generated embeddings =  (1, 376, 256)
Shape of generated embeddings =  (1, 16, 256)
Shape of generated embeddings =  (1, 437, 256)
Shape of generated embeddings =  (1, 43, 256)
Shape of generated embeddings =  (1, 345, 256)
Shape of generated embeddings =  (1, 134, 256)
Shape of generated embed

...

Shape of generated embeddings =  (1, 66, 256)
Shape of generated embeddings =  (1, 457, 256)
Shape of generated embeddings =  (1, 52, 256)
Shape of generated embeddings =  (1, 171, 256)
Shape of generated embeddings =  (1, 302, 256)
Shape of generated embeddings =  (1, 39, 256)
Shape of generated embeddings =  (1, 121, 256)
Shape of generated embeddings =  (1, 33, 256)
Shape of generated embeddings =  (1, 221, 256)
Shape of generated embeddings =  (1, 80, 256)
Shape of generated embeddings =  (1, 185, 256)
Shape of generated embeddings =  (1, 27, 256)
Shape of generated embeddings =  (1, 79, 256)
Shape of generated embeddings =  (1, 171, 256)
Shape of generated embeddings =  (1, 119, 256)
Shape of generated embeddings =  (1, 73, 256)
Shape of generated embeddings =  (1, 276, 256)
Shape of generated embeddings =  (1, 385, 256)
Shape of generated embeddings =  (1, 288, 256)
Shape of generated embeddings =  (1, 33, 256)
Shape of generated embeddings =  (1, 66, 256)
Shape of generated embe

Shape of generated embeddings =  (1, 38, 256)
Shape of generated embeddings =  (1, 100, 256)
Shape of generated embeddings =  (1, 120, 256)
Shape of generated embeddings =  (1, 118, 256)
Shape of generated embeddings =  (1, 73, 256)
Shape of generated embeddings =  (1, 55, 256)
Shape of generated embeddings =  (1, 80, 256)
Shape of generated embeddings =  (1, 28, 256)
Shape of generated embeddings =  (1, 59, 256)
Shape of generated embeddings =  (1, 221, 256)
Shape of generated embeddings =  (1, 81, 256)
Shape of generated embeddings =  (1, 22, 256)
Shape of generated embeddings =  (1, 680, 256)
Shape of generated embeddings =  (1, 72, 256)
Shape of generated embeddings =  (1, 53, 256)
Shape of generated embeddings =  (1, 45, 256)
Shape of generated embeddings =  (1, 134, 256)
Shape of generated embeddings =  (1, 128, 256)
Shape of generated embeddings =  (1, 345, 256)
Shape of generated embeddings =  (1, 57, 256)
Shape of generated embeddings =  (1, 45, 256)
Shape of generated embeddi

Shape of generated embeddings =  (1, 88, 256)
Shape of generated embeddings =  (1, 124, 256)
Shape of generated embeddings =  (1, 68, 256)
Shape of generated embeddings =  (1, 137, 256)
Shape of generated embeddings =  (1, 43, 256)
Shape of generated embeddings =  (1, 22, 256)
Shape of generated embeddings =  (1, 95, 256)
Shape of generated embeddings =  (1, 54, 256)
Shape of generated embeddings =  (1, 149, 256)
Shape of generated embeddings =  (1, 218, 256)
Shape of generated embeddings =  (1, 109, 256)
Shape of generated embeddings =  (1, 58, 256)
Shape of generated embeddings =  (1, 137, 256)
Shape of generated embeddings =  (1, 552, 256)
Shape of generated embeddings =  (1, 554, 256)
Shape of generated embeddings =  (1, 100, 256)
Shape of generated embeddings =  (1, 57, 256)
Shape of generated embeddings =  (1, 89, 256)
Shape of generated embeddings =  (1, 112, 256)
Shape of generated embeddings =  (1, 357, 256)
Shape of generated embeddings =  (1, 456, 256)
Shape of generated emb

Shape of generated embeddings =  (1, 87, 256)
Shape of generated embeddings =  (1, 87, 256)
Shape of generated embeddings =  (1, 34, 256)
Shape of generated embeddings =  (1, 18, 256)
Shape of generated embeddings =  (1, 192, 256)
Shape of generated embeddings =  (1, 107, 256)
Shape of generated embeddings =  (1, 51, 256)
Shape of generated embeddings =  (1, 89, 256)
Shape of generated embeddings =  (1, 79, 256)
Shape of generated embeddings =  (1, 76, 256)
Shape of generated embeddings =  (1, 112, 256)
Shape of generated embeddings =  (1, 16, 256)
Shape of generated embeddings =  (1, 130, 256)
Shape of generated embeddings =  (1, 120, 256)
Shape of generated embeddings =  (1, 68, 256)
Shape of generated embeddings =  (1, 221, 256)
Shape of generated embeddings =  (1, 91, 256)
Shape of generated embeddings =  (1, 98, 256)
Shape of generated embeddings =  (1, 123, 256)
Shape of generated embeddings =  (1, 83, 256)
Shape of generated embeddings =  (1, 56, 256)
Shape of generated embeddin

Shape of generated embeddings =  (1, 332, 256)
Shape of generated embeddings =  (1, 92, 256)
Shape of generated embeddings =  (1, 63, 256)
Shape of generated embeddings =  (1, 118, 256)
Shape of generated embeddings =  (1, 40, 256)
Shape of generated embeddings =  (1, 147, 256)
Shape of generated embeddings =  (1, 16, 256)
Shape of generated embeddings =  (1, 51, 256)
Shape of generated embeddings =  (1, 249, 256)
Shape of generated embeddings =  (1, 98, 256)
Shape of generated embeddings =  (1, 197, 256)
Shape of generated embeddings =  (1, 170, 256)
Shape of generated embeddings =  (1, 83, 256)
Shape of generated embeddings =  (1, 31, 256)
Shape of generated embeddings =  (1, 59, 256)
Shape of generated embeddings =  (1, 29, 256)
Shape of generated embeddings =  (1, 206, 256)
Shape of generated embeddings =  (1, 49, 256)
Shape of generated embeddings =  (1, 90, 256)
Shape of generated embeddings =  (1, 113, 256)
Shape of generated embeddings =  (1, 39, 256)
Shape of generated embeddi

In [15]:
except_test_lst

[]